<a href="https://colab.research.google.com/github/GenAIUnplugged/langchain_series/blob/main/ChatMessageHistory_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install langchain langchain-core langchain-community langchain_openai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00


In [20]:
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import ChatMessageHistory

In [5]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [7]:
# Step 1: Use a structured message history
chat_history = ChatMessageHistory()

In [8]:
# Step 2: Plug that into buffer memory
memory = ConversationBufferMemory(chat_memory=chat_history, return_messages=True)

<ipython-input-8-61d1bff23894>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(chat_memory=chat_history, return_messages=True)


In [9]:
# Step 3: Initialize LLM and chain
llm = ChatOpenAI(temperature=0)  # Use your API key in environment
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

<ipython-input-9-477952510110>:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm, memory=memory, verbose=True)


In [10]:
# Step 4: Simulate a conversation
print(">>> User: Hello!")
print(">>> Bot:", conversation.predict(input="Hello!"))

>>> User: Hello!


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hello!
AI:

> Finished chain.
>>> Bot: Hello! How are you today?


In [11]:
print("\n>>> User: What’s my name?")
print(">>> Bot:", conversation.predict(input="What’s my name?"))


>>> User: What’s my name?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hello!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={})]
Human: What’s my name?
AI:

> Finished chain.
>>> Bot: I'm sorry, I do not have access to your personal information, so I do not know your name.


In [13]:
# Step 5: Inspect stored messages
print("\n🔍 Message history (structured):")
for msg in chat_history.messages:
      print(msg)
    # print(f"{msg.type.upper()}: {msg.content}")


🔍 Message history (structured):
content='Hello!' additional_kwargs={} response_metadata={}
content='Hello! How are you today?' additional_kwargs={} response_metadata={}
content='What’s my name?' additional_kwargs={} response_metadata={}
content="I'm sorry, I do not have access to your personal information, so I do not know your name." additional_kwargs={} response_metadata={}


In [14]:
# Step 6: Clear memory
chat_history.clear()
print("\n🧹 Memory cleared.")


🧹 Memory cleared.


In [15]:
# Confirm it's empty
print("✅ Current messages:", chat_history.messages)

✅ Current messages: []


What’s Happening ??

**ChatMessageHistory** \
Internally stores each message like: \
AIMessage(content="Your name is Sarah.") \
HumanMessage(content="What’s my name?") \

**ConversationBufferMemory** \
Formats it into something like: \
Human: What’s my name? \
AI: Your name is Sarah. \

**Manual Access:** \
chat_history.messages.append(HumanMessage(content="Hi again")) \
chat_history.clear()  # resets all history \

🔁 Use chat_history.messages.pop() to remove the last turn \
🗂️ Save chat history to a file or database \
🤖 Inject initial messages like a system prompt

In [25]:
def chat_fn(user_input,history):
  response = conversation.predict(input=user_input)
  history.append((user_input,response)) #Gradio UI history
  return history,history

# Reset handler
def reset():
    chat_history.clear()
    return [], []

In [26]:
# Gradio UI
with gr.Blocks() as demo:
  gr.Markdown("### 💬 LangChain + Gradio Chatbot with Memory")
  chatbot = gr.Chatbot()
  state = gr.State([])
  user_input = gr.Textbox(placeholder = "Type your message...",show_label = False)

  send_btn = gr.Button("Send")
  clear_btn = gr.Button("Clear Chat")

  send_btn.click(chat_fn, [user_input, state], [chatbot, state])
  user_input.submit(chat_fn, [user_input, state], [chatbot, state])  # support Enter key
  clear_btn.click(fn=reset, outputs=[chatbot, state])

<ipython-input-26-35f726da218e>:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


In [27]:
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3668b7f52ac8dc1c62.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
